In [25]:
import serial
import sys
import time
import cv2
import pytesseract as ocr
import matplotlib.pyplot as plt

from PyQt5.QtCore import (Qt, QThread, pyqtSignal, pyqtSlot,QTimer)
from PyQt5.QtWidgets import (QMainWindow, QAction, qApp, QApplication, QWidget, QPushButton, 
                             QLineEdit, QInputDialog, QTextEdit, QLabel, QVBoxLayout, QHBoxLayout, 
                             QGridLayout)
from PyQt5.QtGui import QIcon, QPainter, QColor, QBrush, QImage, QPixmap
from IPython.display import (Audio, display, clear_output)

In [16]:
sensor_string = {
    "dist" : 0,
    "vermelho" : 0,
    "verde" : 0,
    "azul" : 0,
    "temp" : 0
}
sensor_value = {
    "dist" : 0,
    "vermelho" : 0,
    "verde" : 0,
    "azul" : 0,
    "temp" : 0
}

In [26]:
sock = serial.Serial('/dev/rfcomm0',timeout = 1.5)
cap = cv2.VideoCapture(0)
    
img_indisponivel = cv2.imread("indisponivel.png",0)
img_indisponivel = cv2.cvtColor(img_indisponivel, cv2.COLOR_GRAY2RGB)

h, w, ch = img_indisponivel.shape
bytesPerLine = ch * w
convertToQtFormat = QImage(img_indisponivel.data, w, h, bytesPerLine, QImage.Format_RGB888)
p_indisponivel = convertToQtFormat.scaled(640, 480, Qt.KeepAspectRatio)

    
class Example(QMainWindow):
    
    def __init__(self):
        super().__init__()
        
        self.initUI()
        #self.arduRead()
        self.cameraRead()
        
    def close_window(self):
        #sock.close()
        cap.release()
        self.close()
    
    def disconnect(self):
        # Close socket connection to device
        sock.close()
        
    def led_on(self):
        # Send 'H' which the Arduino detects as turning the light on
        sock.write(b'H')
        

    def led_off(self):
        # Send 'L' to turn off the light attached to the Arduino
        sock.write(b'L')
        
    def cameraRead(self):
        ret, frame = cap.read()
        if ret:
            # https://stackoverflow.com/a/55468544/6622587
            grayImage = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            
            # inserir processamento de OCR e código de barras
            
            h, w, ch = grayImage.shape
            bytesPerLine = ch * w
            convertToQtFormat = QImage(grayImage.data, w, h, bytesPerLine, QImage.Format_RGB888)
            p = convertToQtFormat.scaled(640, 480, Qt.KeepAspectRatio)
            
            # Update screen
            self.label.setPixmap(QPixmap.fromImage(p))
        QTimer.singleShot(20, self.cameraRead)
        
        
    def arduRead(self):
        # Recebe os dados do Arduino e faz seu tratamento 
        sock.reset_input_buffer()
        recv = sock.read(5)
        sensor_string["dist"] = recv[1:5]
        recv = sock.read(4)
        sensor_string["vermelho"] = recv[1:4]
        recv = sock.read(4)
        sensor_string["verde"] = recv[1:4]
        recv = sock.read(4)
        sensor_string["azul"] = recv[1:4]
        
        #Disponibiliza as informações no editor de texto
        print(int(sensor_string["dist"]))
        self.label1.setNum(int(sensor_string["dist"]))
        
        #self.textEdit.clear()
        texto = ("Distancia: " + str(sensor_string["dist"]) + 
                 "\nVermelho: " + str(sensor_string["vermelho"]) +
                 "\nVerde: " + str(sensor_string["verde"]) +
                 "\nAzul: " + str(sensor_string["azul"]))  
        #self.textEdit.setText(texto)    
        QTimer.singleShot(10, self.arduRead)
        
    
    def initUI(self):
        
        haha = "haha"
        
        self.closeAct = QAction(QIcon('exit.png'), 'Close', self)
        self.closeAct.setStatusTip('Fechar janela')
        self.closeAct.triggered.connect(self.close_window)
        self.toolbar1 = self.addToolBar('Fechar janela')
        self.toolbar1.addAction(self.closeAct)
        
        self.disconnectAct = QAction(QIcon('bluetooth.png'), 'Disconnect', self)
        self.disconnectAct.setStatusTip('Desconectar dispositivo')
        self.disconnectAct.triggered.connect(self.disconnect)
        self.toolbar2 = self.addToolBar('Disconnect')
        self.toolbar2.addAction(self.disconnectAct)
        
        self.ledOnAct = QAction(QIcon('led_on.png'), 'Led On', self)
        self.ledOnAct.setStatusTip('Acender LED')
        self.ledOnAct.triggered.connect(self.led_on)
        self.toolbar3 = self.addToolBar('Acender LED')
        self.toolbar3.addAction(self.ledOnAct)
        
        self.ledOffAct = QAction(QIcon('led_off.png'), 'Led Off', self)
        self.ledOffAct.setStatusTip('Apagar LED')
        self.ledOffAct.triggered.connect(self.led_off)
        self.toolbar4 = self.addToolBar('Apagar LED')
        self.toolbar4.addAction(self.ledOffAct)
                      
        # Label de texto:
        self.label1 = QLabel(self)
        self.label1.setText("Teste Label1")
        
        # Label de vídeo:
        self.label = QLabel(self)        
        self.setCentralWidget(self.label)
        # setar dimensões do self.label de texto: ??
        # setar dimensões do self.label de vídeo: 640x480
        
        self.vbox = QVBoxLayout()
        self.vbox.addWidget(self.label1)
        self.setLayout(self.vbox)         
        self.label.setPixmap(QPixmap.fromImage(p_indisponivel))
        
        #self.setLayout(vbox)
        self.statusBar().showMessage('Ready')
        
        self.setGeometry(250, 250, 300, 300)
        self.setWindowTitle('Monitoramento')    
        self.show()
        
        
        
if __name__ == '__main__':
    
    if 'app' in locals():
        del app
        
    app = QApplication(sys.argv)
    ex = Example()
    sys.exit(app.exec_())

SystemExit: 0